In [ ]:
import numpy.random as npr

import jax.numpy as jnp
from jax import jit, grad, random
from jax.experimental import optimizers
from jax.experimental import stax
from jax.experimental.stax import (AvgPool, BatchNorm, Conv, Dense, FanInSum,
                                   FanOut, Flatten, GeneralConv, Identity,
                                   MaxPool, Relu, LogSoftmax)
import flax.linen as nn
from typing import Sequence

In [ ]:
!pip install flax

     |████████████████████████████████| 184kB 30.2MB/s 
     |████████████████████████████████| 122kB 40.6MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 


In [ ]:
!pip install jax-resnet

In [ ]:
from jax_resnet import pretrained_resnest
ResNeSt50, variables = pretrained_resnest(50)
model = ResNeSt50()
out = model.apply(variables,
                  jnp.ones((32, 224, 224, 3)),  # ImageNet sized inputs.
                  mutable=False)  # Ensure `batch_stats` aren't updated.

Downloading: "https://github.com/zhanghang1989/ResNeSt/releases/download/weights_step1/resnest50-528c19ca.pth" to /root/.cache/torch/hub/checkpoints/resnest50-528c19ca.pth


## Get the ResNet Backbone by removing the linear layers 

In [ ]:
model.layers = model.layers[:-1]
out = model.apply(variables,
                  jnp.ones((32, 224, 224, 3)),  # ImageNet sized inputs.
                  mutable=False)  # Ensure `batch_stats` aren't updated.

In [ ]:
out.shape #channel dimension looks good (2048 in DETR paper)

(32, 2048)

## Build the DETR Transformer Encoder/Decoder

In [ ]:
class MLPHead(nn.Module):
  features: Sequence[int]

  @nn.compact
  def __call__(self, x):
    for feat in self.features[:-1]:
      x = nn.relu(nn.Dense(feat)(x))
    x = nn.Dense(self.features[-1])(x)
    return x

mlp_head = MLPHead([12, 8, 4])
mlp_head

MLPHead(
    # attributes
    features = [12, 8, 4]
)

In [ ]:
class DecoderLayer(nn.Module):
  feats: Sequence[int].  #linear layers features 
  decoder_widths: Sequence[int]
  input_shape: Sequence[int]
  activation: string
  droupout: float 
  nhead: int 
  dim_feedforward: int
  normalized_before: False
  d_model: int

  def setup(self):
    input_dim = jnp.prod(jnp.asarray(self.input_shape))

    self.self_attn = nn.MultiHeadDotProductAttention()
    self.multihead_attn = nn.MultiHeadDotProductAttention()

    self.dropout = nn.Drouput(rate=dropout)
    self.linear1 = nn.Dense(feat[0])
    self.dropout1 = nn.Drouput(rate=dropout) 
    self.norm1 =  nn.LayerNorm()
    self.linear2 = nn.Dense(feat[1])
    self.dropout2 = nn.Drouput(rate=dropout) 
    self.norm2 =  nn.LayerNorm()
    self.linear3 = nn.Dense(feat[2])
    self.dropout3 = nn.Drouput(rate=dropout) 
    self.norm3 =  nn.LayerNorm()
    self.nomalize_before = normalize_before

    self.activation 

  def __call__(self, x):
    return self.decode(self.encode(x))

  def encode(self, x):
    assert x.shape[1:] == self.input_shape
    return self.encoder(jnp.reshape(x, (x.shape[0], -1)))

  def decode(self, z):
    z = self.decoder(z)
    x = nn.sigmoid(z)
    x = jnp.reshape(x, (x.shape[0],) + self.input_shape)
    return x




SyntaxError: ignored